In [258]:
from scapy.all import *

In [259]:
pgp_email_file = "data/pgpemail.txt"

In [260]:
with open(pgp_email_file, "r") as f:
    pgp_email_data = f.read()

In [261]:
print pgp_email_data

Return-Path: <erin.lynn.root+caf_=lynn=fastmail.es@gmail.com>
Received: from compute2.internal (compute2.nyi.mail.srv.osa [10.202.2.42])
	 by sloti14d3p1 (Cyrus git2.5+0-git-fastmail-9328) with LMTPA;
	 Thu, 08 Aug 2013 15:09:55 -0400
X-Sieve: CMU Sieve 2.4
X-Spam-known-sender: yes
X-Spam-score: 0.0
X-Spam-hits: BAYES_00 -1.9, RCVD_IN_DNSWL_LOW -0.7, LANGUAGES unknown,
  BAYES_USED global, SA_VERSION 3.3.2
X-Spam-source: IP='74.125.82.177', Host='mail-we0-f177.google.com', Country='US',
  FromHeader='cx', MailFrom='com'
X-Spam-charsets: 
X-Resolved-to: lynn@fastmail.es
X-Delivered-to: lynn@fastmail.es
X-Mail-from: erin.lynn.root+caf_=lynn=fastmail.es@gmail.com
Received: from mx4 ([10.202.2.203])
  by compute2.internal (LMTPProxy); Thu, 08 Aug 2013 15:09:55 -0400
Received: from mail-we0-f177.google.com (mail-we0-f177.google.com [74.125.82.177])
	(using TLSv1 with cipher ECDHE-RSA-RC4-SHA (128/128 bits))
	(No client certificate requested)
	by mx4.messagingengine.com (Postfix) with ESMTPS

In [262]:
pgp_query = 'protocol="application/pgp-encrypted"'

In [263]:
pgp_query in pgp_email_data

True

In [264]:
from email.parser import Parser
import re
import socket as sock

In [265]:
def valid_id(address):
    try:
        sock.inet_aton(address)
        return True
    except:
        return False
    
def filter_ip(address):
    if address.startswith("192.168"):
        return False
    if address.startswith("10"):
        return False
    return True

def parse_email_IP_addrs(email):
    with open(email) as e:
        headers = Parser().parse(e)
    received_trail = (v for k, v in headers.items() if 'Received' in k)
    parsed_headers = [re.findall(r'[0-9]+(?:\.[0-9]+){3}', s) for s in received_trail]

    valid_ips = []
    for ip_addrs in parsed_headers:
        for ip in ip_addrs:
            if valid_id(ip) and filter_ip(ip):
                valid_ips.append(ip)
    return list(set(valid_ips))

In [266]:
ip_addrs = parse_email_IP_addrs(pgp_email_file)

In [267]:
ip_addrs

['74.125.82.177', '66.111.4.26', '5.28.99.177']

In [268]:
import pygeoip

In [269]:
def map_ip(ip_addrs):
    gip = pygeoip.GeoIP('data/GeoLiteCity.dat')
    coordinates = []
    for ip in ip_addrs:
        geo_data = gip.record_by_addr(ip)
        if geo_data:
            lat = geo_data['latitude']
            lon = geo_data['longitude']
            coordinates.append((lon, lat))

    return coordinates

In [270]:
mapped_ips = map_ip(ip_addrs)

In [271]:
mapped_ips

[(-122.0574, 37.41919999999999),
 (-74.0012, 40.7089),
 (13.400000000000006, 52.516699999999986)]

In [272]:
import geojson

In [313]:
def create_points(coordinates):
    points = []
    j = 1
    for coord in coordinates:
        point = {}
        point["type"] = "Point"
        point["coordinates"] = coord
        point["id"] = j
        point["properties"] = {"title": "server %i" % j}
        points.append(point)

    return points


def create_lines(coordinates):
    lines = []
    for index in xrange(0, len(coordinates)-1):
        line = {}
        line["type"] = "LineString"
        line["coordinates"] = [coordinates[index], coordinates[index+1]]
        lines.append(line)
    
    return lines

def create_geojson(coordinates):
    points = create_points(coordinates)
    lines = create_lines(coordinates)
    d = {"type": "GeometryCollection"}
    data = lines + points
    for item in data:
        d.setdefault("geometries", []).append(item)

    return geojson.dumps(d)


In [314]:
data = create_geojson(mapped_ips)

In [315]:
print data

{"type": "GeometryCollection", "geometries": [{"type": "LineString", "coordinates": [[-122.0574, 37.41919999999999], [-74.0012, 40.7089]]}, {"type": "LineString", "coordinates": [[-74.0012, 40.7089], [13.400000000000006, 52.516699999999986]]}, {"type": "Point", "id": 1, "coordinates": [-122.0574, 37.41919999999999], "properties": {"title": "server 1"}}, {"type": "Point", "id": 1, "coordinates": [-74.0012, 40.7089], "properties": {"title": "server 1"}}, {"type": "Point", "id": 1, "coordinates": [13.400000000000006, 52.516699999999986], "properties": {"title": "server 1"}}]}


In [278]:
%%HTML
<script src="http://cdnjs.cloudflare.com/ajax/libs/leaflet/0.7.3/leaflet.js"></script>
<link rel="stylesheet" href="http://cdnjs.cloudflare.com/ajax/libs/leaflet/0.7.3/leaflet.css" />

In [316]:
%%HTML
<script>
var point_data = {"type": "GeometryCollection", "geometries": [{"type": "LineString", "coordinates": [[-122.0574, 37.41919999999999], [-74.0012, 40.7089]]}, {"type": "LineString", "coordinates": [[-74.0012, 40.7089], [13.400000000000006, 52.516699999999986]]}, {"type": "Point", "id": 1, "coordinates": [-122.0574, 37.41919999999999], "properties": {"title": "server 1"}}, {"type": "Point", "id": 1, "coordinates": [-74.0012, 40.7089], "properties": {"title": "server 1"}}, {"type": "Point", "id": 1, "coordinates": [13.400000000000006, 52.516699999999986], "properties": {"title": "server 1"}}]};
</script>

In [319]:
%%HTML

<style>#map { height: 300px; }</style>
<div id="map"></div>
<script>
var map = L.map('map').setView([44.96795, -54.3287], 3);

L.tileLayer('https://{s}.tiles.mapbox.com/v3/{id}/{z}/{x}/{y}.png', {
    maxZoom: 18,
    attribution: 'Map data &copy; <a href="http://openstreetmap.org">OpenStreetMap</a> contributors, ' +
        '<a href="http://creativecommons.org/licenses/by-sa/2.0/">CC-BY-SA</a>, ' +
        'Imagery © <a href="http://mapbox.com">Mapbox</a>',
        id: 'examples.map-20v6611k'
}).addTo(map);

L.geoJson(point_data).addTo(map);


</script>